In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from matplotlib.colors import ListedColormap
from sklearn.tree import export_graphviz

%matplotlib inline

In [33]:
inp = pd.read_csv(r'C:/Users/rohanr730/Desktop/CHEEL/competency_data_master.csv', encoding = 'ISO-8859-1', 
                  usecols = ['New Competency Project', 'Sector', 'Industry', 'Net Revenue', 'Engagement Margin'])
inp.head()

,New Competency Project,Sector,Industry,Net Revenue,Engagement Margin
0,Digital Services,Insurance,Insur - Insurance Service Providers,6.336000e+03,6336.00
1,Digital Services,Services,Svcs - Professional Services / Consultancy,1.034275e+08,30294218.72
2,Digital Services,Technology,Tech - Computers & Networking,2.482383e+06,2266447.28
3,Digital Services,Telecom,T'Com - Basic Telecom,2.982553e+06,2314712.98
4,Digital Services,Telecom,T'Com - Value Added Service Providers (VAS),2.488000e+04,24880.00


In [34]:
inp['EM'] = (inp['Engagement Margin']/inp['Net Revenue']) * 100

In [38]:
inp = inp.drop(['Engagement Margin', 'Net Revenue'], axis = 1)

In [40]:
inp['Sector'].value_counts()

Banking and Capital Markets       450
Technology                        443
Retail & Consumer                 402
Services                          400
Government                        397
Energy - Power & Mining           371
Other Industrial Products         363
Engineering & Construction        285
Pharmaceuticals & Life Science    271
Telecom                           235
Entertainment and Media           227
Automotive                        217
Energy - Oil & Gas                216
Healthcare                        190
Transport,Logistic,Infrastruct    186
Investment Management             182
Insurance                         166
Private Equity                    152
Chemicals                         150
Metals                            126
Education                          98
Aerospace & Defence                54
-                                   8
Real Estate                         4
Entertainment & Media - E&M         1
Name: Sector, dtype: int64

In [54]:
inp['Sector'].sort_values().unique() == 'Insurance'

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [41]:
inp['Sector_dummies'] = inp['Sector'].str.get_dummies().values.tolist()
inp['Industry_dummies'] = inp['Industry'].str.get_dummies().values.tolist()
inp['Project_dummies'] = inp['New Competency Project'].str.get_dummies().values.tolist()

In [43]:
inp['Sector_dummies'][0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [55]:
prep_data = inp.drop(['New Competency Project', 'Sector', 'Industry'], axis = 1)

In [58]:
# Removing negative infinities and NaNs

# Replace both with zero

prep_data = prep_data[~(prep_data['EM'] < -1.0e+25)]
prep_data = prep_data.dropna()

In [62]:
dataset = prep_data.values

In [64]:
dataset[0]

array([100.0,
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
      dtype=o

In [9]:
X = dataset[:,1:4]
y = dataset[:,0]

In [10]:
n_estimators = 1
model = RandomForestClassifier(n_estimators = n_estimators)
clf = clone(model)

In [65]:
X_new = []
for x in X:
    
    l = x[0] + x[1] + x[2]
    X_new.append(l)

In [12]:
X_train = np.array(X_new)

In [13]:
lower = -1.0e+2
upper = 1.0e+2
max_y = max(y)
min_y = min(y)
y_norm = [((upper - lower)/(max_y - min_y)) * x for x in y]

In [14]:
print(min(y_norm))
print(max(y_norm))

-144.31881944437762
55.68118055562237


In [68]:
y_train

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [15]:
y_train = np.array(y_norm, dtype = np.int64)

In [16]:
for ix in range(len(y_train)):
    
    if y_train[ix] < 0:
        y_train[ix] = 0
    elif y_train[ix] >= 0:
        y_train[ix] = 1

In [19]:
print(max(y_train))
print(min(y_train))

1
0


In [28]:
np.savetxt("x_train.csv", X_train, delimiter = ",")
np.savetxt("y_train.csv", y_train, delimiter = ",")

In [20]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
feature_names = []
for ix in range(len(X_train[0])):
    name = 'Feature' + str(ix)
    feature_names.append(name)
target_names = ['Synergic', 'NonHarmonic']

In [26]:
estimator = model.estimators_[0]

# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

FileNotFoundError: [WinError 2] The system cannot find the file specified